<a href="https://colab.research.google.com/github/hoangtheanhhp/ZaloQA/blob/bert_lstm/QASystem/run_zalo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!rm -rf zalo
!test -d zalo || git clone https://github.com/hoangtheanhhp/ZaloQA zalo
import sys
if 'zalo' not in sys.path:
  sys.path+=['zalo']

Cloning into 'zalo'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 569 (delta 25), reused 32 (delta 11), pack-reused 516
Receiving objects: 100% (569/569), 94.16 MiB | 20.28 MiB/s, done.
Resolving deltas: 100% (253/253), done.
Checking out files: 100% (37/37), done.


In [2]:
%tensorflow_version 1.x
!pip install tensorflow==1.15.2
!pip install kashgari==1.1.4



TensorFlow is already loaded. Please restart the runtime to change versions.
     |████████████████████████████████| 110.5MB 99kB/s 
     |████████████████████████████████| 512kB 36.6MB/s 
     |████████████████████████████████| 3.8MB 36.1MB/s 
ERROR: tensorflow-gpu 2.1.0 has requirement tensorboard<2.2.0,>=2.1.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow-gpu 2.1.0 has requirement tensorflow-estimator<2.2.0,>=2.1.0rc0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow 2.2.0rc3
    Uninstalling tensorflow-2.2.0rc3:
      Successfully uninstalled tensorflow-2.2.0rc3


In [0]:
DATASET_PATH = "zalo/Dataset/zalo"
SAVE_MODEL_PATH = "/content/drive/My Drive/ZaloAI/result/model.h5"
MODEL_PATH = "/content/drive/My Drive/ZaloAI/bert"

In [3]:
from zalo.QASystem.preprocess import ZaloDatasetProcessor
import kashgari
from kashgari.embeddings import BERTEmbedding

bert_embedding = BERTEmbedding(MODEL_PATH,
                               task=kashgari.CLASSIFICATION,
                               sequence_length=128)

tokenizer = bert_embedding.tokenizer
processor = ZaloDatasetProcessor()
processor.load_from_path(DATASET_PATH, "train")
processor.load_from_path(DATASET_PATH, "val")

data = processor.train_data
val = processor.val_data
train_x = []
train_y = []
val_x = []
val_y = []
for d in data:
  train_x.append(tokenizer.tokenize(d.get('question')) + 
                 tokenizer.tokenize(d.get('text'))[1:])
  train_y.append(d.get('label', False))

for d in val:
  val_x.append(tokenizer.tokenize(d.get('question')) + 
               tokenizer.tokenize(d.get('text'))[1:])
  val_y.append(d.get('label', False))

print(len(train_x), len(train_y), len(val_x), len(val_y))


/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')
100%|██████████| 3906/3906 [00:00<00:00, 644389.22it/s]


[{'question': 'Ngô Quyền sinh vào năm nào', 'text': 'Những suy đoán về tuổi tác của Ngô Quyền, Ngô Xương Ngập và con cả Xương Ngập là Ngô Xương Tỷ (tức Khuông Việt đại sư) cho thấy bà không phải là người sinh ra Ngô Xương Ngập còn vì Xương Ngập ra đời khoảng năm 915 khi Ngô Quyền chưa gặp bà và hơn Xương Văn khá nhiều tuổi.', 'label': False}, {'question': 'Trái đất có bao nhiêu nước', 'text': 'Khối lượng nước trong các đại dương xấp xỉ 1,35 tấn, hoặc khoảng 1/4400 khối lượng của Trái Đất, và chiếm thể tích 1,386 km3. Nếu tất cả đất trên Trái Đất được trải phẳng ra, mực nước biển sẽ dâng lên cao hơn 2,7 km. Khoảng 97,5% nước có chứa muối, còn lại 2,5% là nước ngọt và phần lớn nước ngọt, khoảng 68,7%, đang ở dạng băng.', 'label': True}, {'question': 'Thủ lĩnh tối cao Phan Đình Phùng từng thi đỗ danh hiệu gì', 'text': 'Năm 1876, Phan Đình Phùng đỗ cử nhân. Năm sau (1877), ông thi đậu Đình nguyên Tiến sĩ, được bổ làm tri huyện Yên Khánh (tỉnh Ninh Bình). Tại triều, ông tố cáo nhiều vụ khuấ

In [4]:
from tensorflow.python import keras
from kashgari.tasks.classification import BiLSTM_Model
from kashgari.callbacks import EvalCallBack

import logging
logging.basicConfig(level='DEBUG')

model = BiLSTM_Model()


model.fit(train_x,
          train_y,
          val_x,
          val_y,
          epochs=10, batch_size=100)

model.save(SAVE_MODEL_PATH)


Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 176)]             0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 176, 100)          720500    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               234496    
_________________________________________________________________
dense (Dense)                (None, 2)                 514       
Total params: 955,510
Trainable params: 955,510
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
143/143 [==============================] - 288s 2s/step - loss: 0.6084 - acc: 0.6958 - val_loss: 0.6256 - val_acc: 0.6544
Epoch 2/10
143/143 [==============================] - 282s 2s/step - loss: 0.5460 - acc: 0.7302 - val_loss: 0.6082 - v

In [14]:
processor.load_from_path("/content/zalo/Dataset/zalo","test")
test = processor.test_data
test_x = []
test_y = []
for d in test:
  text_x.append(tokenizer.tokenize(d.get('question')) + 
               tokenizer.tokenize(d.get('text'))[1:])
  text_y.append(d.get('label'))
# evaluate the model
model.evaluate(test_x, test_y)
# print("loss: %f\n accruracy: %f\nf1_score: %f\nprecision: %f\n, recall: %f\n:" 
#       % (loss, accurany,f1_score,precision, recall))

JSONDecodeError: ignored